In [1]:
import pandas as pd
from urllib import parse
import sqlite3

con = sqlite3.connect("database/db.sqlite")

In [2]:
#con.close()

In [3]:
def get_info_dict(linha):
    url, device_id, referer = linha.split('|')
    url, device_id, referer = url.split('://')[1].strip(), device_id.split(':')[1].strip(), ''.join(referer.split(':')[1:]).strip()
    referer = 'facebook' if 'facebook' in referer else 'google' if 'google' in referer else referer
    
    params_dict = parse.parse_qs(parse.urlparse(url).query)
    for key in params_dict.keys():
        params_dict[key] = params_dict[key][0]

    info_dict = {
        'url':url,
        'device_id':device_id,
        'referer':referer
    }
    return {**info_dict, **params_dict}

def parse_page_view_to_df():
    info_dict = []
    with open(dataset_folder+page_view_path,'r') as page_file:
        for linha in page_file:        
            info_dict.append(get_info_dict(linha))
    return pd.DataFrame(info_dict)

In [4]:
dataset_folder = 'datasets/'
google_path = 'google_ads_media_costs.jsonl'
facebook_path = 'facebook_ads_media_costs.jsonl'
page_view_path = 'pageview.txt'

col_names = ['device_id', 'lead_id', 'registered_at', 'credit_decision', 'credit_decision_at', 'signed_at', 'revenue']
customer_funnel_path = 'customer_leads_funnel.csv'

In [5]:
google_df = pd.read_json(dataset_folder+google_path, lines=True)
facebook_df = pd.read_json(dataset_folder+facebook_path, lines=True)
customer_funnel_df = pd.read_csv(dataset_folder+customer_funnel_path)
page_view_df = parse_page_view_to_df()

In [6]:
google_df['origin'] = 'google'
facebook_df['origin'] = 'facebook'

In [7]:
google_df.to_sql('google_ads',con, if_exists='replace')
facebook_df.to_sql('facebook_ads',con, if_exists='replace')
customer_funnel_df.to_sql('custome_leads_funnel',con, if_exists='replace')
page_view_df.to_sql('page_view',con, if_exists='replace')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [8]:
df = pd.read_sql_query('''
    SELECT campaign_id, campaign_name, origin, sum(cost) AS total_campaign_cost  FROM 
    (
        SELECT cost, google_campaign_id AS campaign_id, google_campaign_name AS campaign_name, origin from google_ads 
        UNION ALL 
        SELECT cost, facebook_campaign_id, facebook_campaign_name, origin from facebook_ads
    ) 
    GROUP BY campaign_id, origin
    ORDER BY total_campaign_cost DESC
    LIMIT 1
''', con)

print('Most expensive campaign: ',df.campaign_name[0],'\nCost: ',df.total_campaign_cost[0])

Most expensive campaign:  creditas|home|natal2018 
Cost:  15626.58


In [10]:
pd.read_sql_query('''
    SELECT * from page_view
    WHERE campaign_id IS NOT NULL
    GROUP BY campaign_id, referer, device_id
    ORDER BY campaign_id ASC
    LIMIT 5
''', con)

,index,ad_creative_id,campaign_id,device_id,referer,url
0,13,20003,1001,0GGtWBPUOB,google,www.creditas.com.br/emprestimo?ad_creative_id=...
1,61,20007,1001,0ORagCcwX1,google,www.bankfacil.com.br/emprestimo-com-garantia?a...
2,100,20006,1001,4OpYhImeuy,google,www.creditas.com.br/emprestimo?ad_creative_id=...
3,208,20007,1001,9GAnXZsCze,google,www.creditas.com.br/conversion?ad_creative_id=...
4,263,20006,1001,D94rOs6neZ,google,www.bankfacil.com.br/emprestimo-com-garantia?a...


In [72]:
pd.read_sql_query('''
    SELECT device_id, sum(revenue) as total_revenue from customer_leads_funnel
    WHERE device_id = '4OpYhImeuy'
    GROUP BY device_id
''', con)

,device_id,total_revenue
0,4OpYhImeuy,398.21
